In [57]:
import sys
import scipy.signal.signaltools as sigtool
from scipy import signal
from scipy.signal import lfilter
import itertools
import scipy.io.wavfile as wf
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

mark_f = 3125 ## 320 microseconds
space_f = 1562.5 ## 640 microseconds

def fsk_demodulator(fileName):

# read the wave file

    fs, sig = wf.read(fileName)


# Bandpass filter mark and space frequencies:

    center_freq = (mark_f + space_f)/2 
    
    trans_width = 260    # Width of transition from pass band to stop band, Hz (assume for now.But must be calculated from Spectrum)

    numtaps = 10        # Size of the FIR filter.

#1. Bandpass filter for mark frequency:

    band_1 = [center_freq, 2*mark_f - center_freq]  # Desired pass band, Hz

    edges_1 = [0, band_1[0] - trans_width, band_1[0], band_1[1],band_1[1] + trans_width, 0.5*fs]

    band_pass_1 = signal.remez(numtaps, edges_1, [0, 1, 0], Hz=fs)

    mark_filtered = signal.lfilter(band_pass_1, 1, sig)

#2. Bandpass filter for space frequency:

    band_2 = [2*space_f - center_freq, center_freq]  # Desired pass band, Hz

    edges_2 = [0, band_2[0] - trans_width, band_2[0], band_2[1],band_2[1] + trans_width, 0.5*fs]

    band_pass_2 = signal.remez(numtaps, edges_2, [0, 1, 0], Hz=fs)

    space_filtered = signal.lfilter(band_pass_2, 1, sig)


#3. Envelope detector on mark and space bandpass filtered signals:

    mark_env = np.abs(sigtool.hilbert(mark_filtered))
    space_env = np.abs(sigtool.hilbert(space_filtered))


#4. Compare and decision:

    seq = range(0, len(mark_env))
    rx = []
    for i in seq:
        if (mark_env[i] > space_env[i]).any():
            rx.append(1)
        if (mark_env[i] < space_env[i]).any():
            rx.append(0)
        if (mark_env[i] == space_env[i]).any():
            rx.append(0)
            
#5. Convert bit to byte:
            
    bytes = [sum([byte[b] << b for b in range(0,8)])
            for byte in zip(*(iter(rx),) * 8)
        ] 
    
#6. Converting bytearray to hexadecimal string 
    res = ','.join(format(x, '02x') for x in bytes) 

    return res

In [58]:
hexa = fsk_demodulator('file_3.wav')
print(len(hexa))
print(hexa)

266348
15,51,69,55,aa,52,57,a9,aa,b2,6a,aa,aa,5a,a9,83,6a,ab,56,55,a5,4a,5b,a9,8a,6a,45,11,52,aa,29,55,c5,75,56,a9,a4,5a,55,a5,4a,95,aa,52,b7,48,a4,95,54,54,89,94,aa,2a,55,4a,aa,6c,75,59,69,4d,ad,aa,ca,94,4a,b5,8a,6a,55,25,a9,26,a2,ba,55,aa,55,4b,55,25,a5,55,57,55,a5,a9,56,53,4a,ad,28,a7,ca,6d,4b,ab,b6,50,55,55,ad,7a,a9,4a,55,ad,ab,44,94,aa,26,cd,aa,56,da,da,aa,1b,25,ae,22,45,51,4a,55,4d,45,85,8a,22,49,55,82,aa,2a,15,89,aa,ac,4a,9b,54,55,56,73,36,4a,55,95,54,a1,4a,55,b4,52,55,11,31,29,6d,a5,94,d6,b4,48,4a,6d,2a,ab,9a,55,a9,8a,b6,aa,56,59,01,55,f1,95,ec,ea,56,69,6f,55,5d,5d,55,ab,b5,6a,56,95,4a,55,6a,97,ea,56,55,d5,a4,a5,20,55,b3,55,2d,55,ad,52,50,ae,6a,a5,95,42,55,b5,76,49,a9,ba,29,57,c6,55,69,6b,6a,8a,aa,52,aa,aa,ca,ab,04,55,55,49,4a,53,4d,a5,40,42,47,45,b5,4a,6c,75,b1,52,a8,58,fa,b6,aa,14,d5,aa,2a,5d,b5,a8,92,b6,6a,55,5d,7a,4a,6a,55,6d,ad,6a,b5,aa,d5,ac,2a,aa,d2,25,91,29,a7,5a,41,a5,4a,a6,28,55,a8,a8,aa,ab,aa,54,a9,aa,ad,a5,25,ad,a8,5a,41,55,92,5a,49,c5,2a,6a,a9,4c,49,55,a1,8a,aa,89,